In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.metrics import *

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[3], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [8]:
def summarize_results(accuracies,precisions,recalls,aucs):
    m, s = mean(accuracies), std(accuracies)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(precisions), std(precisions)
    print( ' Precision: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(recalls), std(recalls)
    print( ' Recall: %.3f%% (+/-%.3f) ' % (m, s))
    m, s = mean(aucs), std(aucs)
    print( ' AUC: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 9s 160ms/step - loss: 3.1730 - accuracy: 0.5356 - precision: 0.5070 - recall: 0.3388 - auc: 0.5642
Epoch 2/30
58/58 [==============================] - 9s 159ms/step - loss: 2.1605 - accuracy: 0.6013 - precision: 0.5860 - recall: 0.5093 - auc: 0.6515
Epoch 3/30
58/58 [==============================] - 10s 175ms/step - loss: 1.7069 - accuracy: 0.6835 - precision: 0.6675 - recall: 0.6472 - auc: 0.7400
Epoch 4/30
58/58 [==============================] - 10s 176ms/step - loss: 1.4562 - accuracy: 0.7185 - precision: 0.7221 - recall: 0.6495 - auc: 0.7976
Epoch 5/30
58/58 [==============================] - 10s 179ms/step - loss: 1.2822 - accuracy: 0.7689 - precision: 0.7304 - recall: 0.8037 - auc: 0.8303
Epoch 6/30
58/58 [==============================] - 10s 176ms/step - loss: 1.1813 - accuracy: 0.7700 - precision: 0.7243 - recall: 0.8224 - auc: 0.8397
Epoch 7/30
58/58 [==============================] - 10s 179ms/step - loss: 1.0523 - accura

58/58 [==============================] - 10s 181ms/step - loss: 0.7401 - accuracy: 0.7766 - precision_1: 0.7843 - recall_1: 0.7220 - auc_1: 0.7886
Epoch 24/30
58/58 [==============================] - 10s 179ms/step - loss: 0.6473 - accuracy: 0.8204 - precision_1: 0.7946 - recall_1: 0.8318 - auc_1: 0.8494
Epoch 25/30
58/58 [==============================] - 10s 180ms/step - loss: 1.0089 - accuracy: 0.5827 - precision_1: 0.5781 - recall_1: 0.4065 - auc_1: 0.5940
Epoch 26/30
58/58 [==============================] - 10s 181ms/step - loss: 0.9317 - accuracy: 0.5915 - precision_1: 0.5779 - recall_1: 0.4766 - auc_1: 0.5897
Epoch 27/30
58/58 [==============================] - 11s 181ms/step - loss: 0.8982 - accuracy: 0.6002 - precision_1: 0.5908 - recall_1: 0.4790 - auc_1: 0.5969
Epoch 28/30
58/58 [==============================] - 10s 181ms/step - loss: 0.8716 - accuracy: 0.6166 - precision_1: 0.6161 - recall_1: 0.4836 - auc_1: 0.6109
Epoch 29/30
58/58 [==============================] - 10s 1

58/58 [==============================] - 11s 181ms/step - loss: 0.8033 - accuracy: 0.7842 - precision_3: 0.7352 - recall_3: 0.8435 - auc_3: 0.8536
Epoch 14/30
58/58 [==============================] - 10s 179ms/step - loss: 0.7603 - accuracy: 0.8094 - precision_3: 0.7749 - recall_3: 0.8364 - auc_3: 0.8595
Epoch 15/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7179 - accuracy: 0.8248 - precision_3: 0.7965 - recall_3: 0.8411 - auc_3: 0.8829
Epoch 16/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7702 - accuracy: 0.7568 - precision_3: 0.7994 - recall_3: 0.6425 - auc_3: 0.8368
Epoch 17/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7377 - accuracy: 0.7755 - precision_3: 0.7942 - recall_3: 0.7033 - auc_3: 0.8481
Epoch 18/30
58/58 [==============================] - 10s 180ms/step - loss: 0.7309 - accuracy: 0.7831 - precision_3: 0.7468 - recall_3: 0.8131 - auc_3: 0.8387
Epoch 19/30
58/58 [==============================] - 10s 1

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation= 'sigmoid', kernel_regularizer=l2(0.01),bias_regularizer=l2(0.01) ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy',Precision(),Recall(),AUC()])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy,precision,recall,auc = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy,precision,recall,auc

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    accuracies = list()
    precisions = list()
    recalls = list()
    aucs = list()
    for r in range(repeats):
        accuracy,precision,recall,auc = evaluate_model_2(trainX, trainy, testX, testy)
        accuracy = accuracy * 100.0
        precision = precision * 100.0
        recall = recall * 100.0
        auc = auc * 100.0
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        aucs.append(auc)
    # summarize results
    summarize_results(accuracies,precisions,recalls,aucs)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 19s 320ms/step - loss: 4.4369 - accuracy: 0.5520 - precision_5: 0.5404 - recall_5: 0.2967 - auc_5: 0.5877
Epoch 2/30
58/58 [==============================] - 18s 315ms/step - loss: 2.6620 - accuracy: 0.6265 - precision_5: 0.6390 - recall_5: 0.4673 - auc_5: 0.6899
Epoch 3/30
58/58 [==============================] - 18s 315ms/step - loss: 2.0245 - accuracy: 0.6911 - precision_5: 0.6560 - recall_5: 0.7173 - auc_5: 0.7491
Epoch 4/30
58/58 [==============================] - 18s 314ms/step - loss: 1.7125 - accuracy: 0.7547 - precision_5: 0.7073 - recall_5: 0.8131 - auc_5: 0.8028
Epoch 5/30
58/58 [==============================] - 18s 315ms/step - loss: 1.5441 - accuracy: 0.7481 - precision_5: 0.7230 - recall_5: 0.7500 - auc_5: 0.8148
Epoch 6/30
58/58 [==============================] - 18s 314ms/step - loss: 1.3781 - accuracy: 0.7930 - precision_5: 0.7661 - recall_5: 0.8037 - auc_5: 0.8536
Epoch 7/30
58/58 [==============================] - 

58/58 [==============================] - 18s 315ms/step - loss: 0.7707 - accuracy: 0.7382 - precision_6: 0.9056 - recall_6: 0.4930 - auc_6: 0.7469
Epoch 23/30
58/58 [==============================] - 18s 315ms/step - loss: 0.7438 - accuracy: 0.7536 - precision_6: 0.8441 - recall_6: 0.5818 - auc_6: 0.7881
Epoch 24/30
58/58 [==============================] - 18s 316ms/step - loss: 0.7381 - accuracy: 0.7459 - precision_6: 0.7593 - recall_6: 0.6706 - auc_6: 0.8288
Epoch 25/30
58/58 [==============================] - 18s 316ms/step - loss: 0.6507 - accuracy: 0.8116 - precision_6: 0.8441 - recall_6: 0.7336 - auc_6: 0.8829
Epoch 26/30
58/58 [==============================] - 18s 311ms/step - loss: 0.6369 - accuracy: 0.8171 - precision_6: 0.7831 - recall_6: 0.8435 - auc_6: 0.8868
Epoch 27/30
58/58 [==============================] - 18s 317ms/step - loss: 0.6748 - accuracy: 0.8018 - precision_6: 0.7763 - recall_6: 0.8107 - auc_6: 0.8561
Epoch 28/30
58/58 [==============================] - 18s 3

58/58 [==============================] - 19s 320ms/step - loss: 0.8816 - accuracy: 0.8116 - precision_8: 0.7783 - recall_8: 0.8364 - auc_8: 0.8662
Epoch 14/30
58/58 [==============================] - 18s 317ms/step - loss: 0.8483 - accuracy: 0.8138 - precision_8: 0.7780 - recall_8: 0.8435 - auc_8: 0.8549
Epoch 15/30
58/58 [==============================] - 18s 318ms/step - loss: 0.8502 - accuracy: 0.7963 - precision_8: 0.8399 - recall_8: 0.6986 - auc_8: 0.8562
Epoch 16/30
58/58 [==============================] - 18s 315ms/step - loss: 0.8977 - accuracy: 0.7777 - precision_8: 0.7635 - recall_8: 0.7617 - auc_8: 0.8422
Epoch 17/30
58/58 [==============================] - 18s 317ms/step - loss: 0.8084 - accuracy: 0.8138 - precision_8: 0.7804 - recall_8: 0.8388 - auc_8: 0.8716
Epoch 18/30
58/58 [==============================] - 18s 317ms/step - loss: 0.7643 - accuracy: 0.8193 - precision_8: 0.7828 - recall_8: 0.8505 - auc_8: 0.8775
Epoch 19/30
58/58 [==============================] - 19s 3